### 读取数据

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:
# 读取数据test数据集
dpath = "../Data/"
testTemp = pd.read_csv(dpath+"test.csv", iterator=True)
# 设置分块参数
loop = True
chunkSize = 100000
chunks = []

while loop:
    try:
        chunk = testTemp.get_chunk(chunkSize)
        chunks.append(chunk)
    except StopIteration:
        loop = False
        print("Iteration is stopped.")
test = pd.concat(chunks, ignore_index=True)
print(len(test))

Iteration is stopped.
2556790


In [3]:
# 读取songs数据集
songs = pd.read_csv(dpath+"songs.csv")
print(len(songs))

2296320


In [4]:
test_songs = pd.merge(test, songs, how="outer")
test_songs = test_songs.drop(["id"], axis=1)
test_songs.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,song_length,genre_ids,artist_name,composer,lyricist,language
0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library,224130.0,458,梁文音 (Rachel Liang),Qi Zheng Zhang,NaN,3.0
1,mp5bRgC7VAhEpWlAiOn4jG8CD9FJZt7725f93t5i99M=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Search,song,224130.0,458,梁文音 (Rachel Liang),Qi Zheng Zhang,NaN,3.0
2,NSBR1BrOO3XDapGQCMi19nRidHD4qzoEXbGGnBap/e0=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-playlist,224130.0,458,梁文音 (Rachel Liang),Qi Zheng Zhang,NaN,3.0
3,kvRxHi514EY2/4FzahC/vBf/Zn13/3ZwXweeepjewzk=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-playlist,224130.0,458,梁文音 (Rachel Liang),Qi Zheng Zhang,NaN,3.0
4,Qh0RP1Refm1D6eV5PjusX+agjnNczGUR8eeWBGj3UvU=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-playlist,224130.0,458,梁文音 (Rachel Liang),Qi Zheng Zhang,NaN,3.0


In [5]:
test_songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4628371 entries, 0 to 4628370
Data columns (total 11 columns):
msno                  object
song_id               object
source_system_tab     object
source_screen_name    object
source_type           object
song_length           float64
genre_ids             object
artist_name           object
composer              object
lyricist              object
language              float64
dtypes: float64(2), object(9)
memory usage: 423.7+ MB


In [6]:
print(test_songs.isnull().sum())

msno                  2071581
song_id                     0
source_system_tab     2080023
source_screen_name    2234464
source_type           2078878
song_length                25
genre_ids              132345
artist_name                25
composer              1595714
lyricist              3008577
language                   42
dtype: int64


### 清理数目比较少的空值

In [7]:
filterData = ['song_length', 'artist_name', 'language']
TestSongs = test_songs.dropna(subset = filterData)
TestSongs = TestSongs.drop(["composer", "lyricist"], axis=1) #其实测试数据中composer缺失值不算多
print(TestSongs.isnull().sum())
len(TestSongs)

msno                  2071581
song_id                     0
source_system_tab     2080023
source_screen_name    2234461
source_type           2078878
song_length                 0
genre_ids              132320
artist_name                 0
language                    0
dtype: int64


4628329

### 将数目比较少的变量进行合并

In [8]:
TestSongs["genre_ids"].value_counts()

465                   1693901
458                    389468
1609                   269344
921                    248757
2022                   211722
958                    190868
2122                   160782
1259                   156975
444                    100642
359                     84288
139                     56805
786                     52985
1152                    50717
1011                    43488
726                     42070
437                     37446
451                     37329
940                     35227
139|125|109             27481
1572|275                25751
1616                    22533
1955                    21279
947                     20923
691                     20103
873                     18783
388                     17263
829                     15362
242                     14234
880                     13428
423                     13305
                       ...   
880|444                     1
1138|275|1572               1
465|139|38

In [9]:
splitGenre = TestSongs.drop('genre_ids', axis=1).join(
    TestSongs['genre_ids'].str.split('|', expand=True).stack().reset_index(
        level=1, drop=True).rename('genre_ids'))

In [10]:
splitGenre["genre_ids"].value_counts()[:50]

465     1752294
458      411261
1609     296378
921      278732
2022     223896
958      198400
1259     174276
2122     172732
444      115921
139       97692
359       91084
786       76038
1152      68628
940       47442
1011      46103
726       45297
1616      44972
947       39152
109       39120
437       37810
451       37442
1572      28866
125       28747
388       28175
275       27358
873       22084
1955      21737
691       20927
829       17894
880       16125
242       15326
423       13801
1180      13229
2086      12397
2157      12332
2130      11692
1138      11452
430       11401
2058      10627
843       10135
850       10134
864       10115
798        9816
374        9626
857        9354
352        8933
893        8542
1995       8134
2072       7461
409        6927
Name: genre_ids, dtype: int64

In [11]:
print(splitGenre.isnull().sum())
print(splitGenre["source_system_tab"].value_counts())
print(splitGenre["source_type"].value_counts())
print(splitGenre["language"].value_counts())
TestSongs["source_screen_name"].value_counts()

msno                  2271330
song_id                     0
source_system_tab     2280158
source_screen_name    2443033
source_type           2278917
song_length                 0
artist_name                 0
language                    0
genre_ids              132320
dtype: int64
my library      1071169
discover         929733
search           293240
radio            227059
listen with      101785
explore           70644
notification       2205
settings            656
Name: source_system_tab, dtype: int64
online-playlist           827812
local-library             608982
local-playlist            310383
radio                     229511
album                     208874
top-hits-for-artist       188335
song                      138325
song-based-playlist        90358
listen-with                87109
topic-article-playlist      5514
my-daily-playlist           2070
artist                       459
Name: source_type, dtype: int64
 52.0    2129463
 3.0     1420449
-1.0      785047
 31.0   

Local playlist more     845098
Online playlist more    529806
Radio                   211193
Album more              176123
Search                  121979
Artist more             110999
Discover Feature         93401
Others profile more      90457
Discover Chart           78999
Discover Genre           41616
Explore                  27871
My library               25557
Unknown                  23620
Discover New              5277
Search Trends             4883
Search Home               4705
My library_Search         2114
Self profile more          131
People local                13
Concert                     13
Payment                     12
People global                1
Name: source_screen_name, dtype: int64

In [12]:
col_origin = splitGenre.columns
otherSouScre = ["People local", "Concert", "Self profile more", "My library_Search",
                "Payment", "People global"]
cols = np.array(splitGenre).tolist()
for col in cols:
    if col[3] in otherSouScre:
        col[3] = "Other"
TestSongs = pd.DataFrame(columns = col_origin, data = cols)

In [13]:
print(TestSongs["source_system_tab"].value_counts())
print(TestSongs["source_type"].value_counts())
TestSongs.isnull().sum()

my library      1071169
discover         929733
search           293240
radio            227059
listen with      101785
explore           70644
notification       2205
settings            656
Name: source_system_tab, dtype: int64
online-playlist           827812
local-library             608982
local-playlist            310383
radio                     229511
album                     208874
top-hits-for-artist       188335
song                      138325
song-based-playlist        90358
listen-with                87109
topic-article-playlist      5514
my-daily-playlist           2070
artist                       459
Name: source_type, dtype: int64


msno                  2271330
song_id                     0
source_system_tab     2280158
source_screen_name    2443033
source_type           2278917
song_length                 0
artist_name                 0
language                    0
genre_ids              132320
dtype: int64

In [14]:
feature_count = TestSongs['genre_ids'].value_counts()  
l = len(feature_count.values)
small_list=[]  # 将样本数少于10000的变量放入list
for i in range(l):
    if feature_count.values[i] <= 6500:
        small_list.append(feature_count.index[i])

    
m = len(TestSongs)
feature_values = TestSongs["genre_ids"].values
for j in range(m):
    for i in small_list:
        if i==feature_values[j]:
            feature_values[j] = "Other"
TestSongs["genre_ids"]=feature_values

In [15]:
print(TestSongs["genre_ids"].value_counts())
print(TestSongs["source_system_tab"].value_counts())
print(TestSongs["source_type"].value_counts())
print(TestSongs["source_screen_name"].value_counts())
print(TestSongs["language"].value_counts())

465      1752294
458       411261
1609      296378
921       278732
2022      223896
958       198400
1259      174276
2122      172732
Other     126312
444       115921
139        97692
359        91084
786        76038
1152       68628
940        47442
1011       46103
726        45297
1616       44972
947        39152
109        39120
437        37810
451        37442
1572       28866
125        28747
388        28175
275        27358
873        22084
1955       21737
691        20927
829        17894
880        16125
242        15326
423        13801
1180       13229
2086       12397
2157       12332
2130       11692
1138       11452
430        11401
2058       10627
843        10135
850        10134
864        10115
798         9816
374         9626
857         9354
352         8933
893         8542
1995        8134
2072        7461
409         6927
Name: genre_ids, dtype: int64
my library      1071169
discover         929733
search           293240
radio            227059
listen 

### Onehot编码

In [16]:
"""
TestSongs["source_system_tab"].astype("object")
TestSongs["source_type"].astype("object")
TestSongs["source_screen_name"].astype("object")
TestSongs[ "genre_ids"].astype("object")
TestSongs["language"].astype("object")
tabCat = pd.get_dummies(TestSongs["source_system_tab"], prefix="source_system_tab")
typeCat = pd.get_dummies(TestSongs["source_type"], prefix="source_type")
nameCat = pd.get_dummies(TestSongs["source_screen_name"], prefix="source_screen_name")
geidsCat = pd. get_dummies(TestSongs["genre_ids"], prefix="genre_ids")
langCat = pd.get_dummies(TestSongs["language"], prefix="language")
one_TestSongs = [TestSongs, tabCat,typeCat, nameCat, geidsCat, langCat]
TestSongs = pd.concat(one_TestSongs, axis=1).drop(["source_system_tab", "source_type", "source_screen_name",
                                                     "genre_ids", "language"], axis=1)
TestSongs.head()
"""

'\nTestSongs["source_system_tab"].astype("object")\nTestSongs["source_type"].astype("object")\nTestSongs["source_screen_name"].astype("object")\nTestSongs[ "genre_ids"].astype("object")\nTestSongs["language"].astype("object")\ntabCat = pd.get_dummies(TestSongs["source_system_tab"], prefix="source_system_tab")\ntypeCat = pd.get_dummies(TestSongs["source_type"], prefix="source_type")\nnameCat = pd.get_dummies(TestSongs["source_screen_name"], prefix="source_screen_name")\ngeidsCat = pd. get_dummies(TestSongs["genre_ids"], prefix="genre_ids")\nlangCat = pd.get_dummies(TestSongs["language"], prefix="language")\none_TestSongs = [TestSongs, tabCat,typeCat, nameCat, geidsCat, langCat]\nTestSongs = pd.concat(one_TestSongs, axis=1).drop(["source_system_tab", "source_type", "source_screen_name",\n                                                     "genre_ids", "language"], axis=1)\nTestSongs.head()\n'

### 将Test_Songs保存起来

In [17]:
TestSongs.to_csv(dpath +'TestSongs.csv',index=False,header=True)